# Grouped Zonal Statistics - Summarizing classes of data within a boundary

## Setup

In [ ]:
import ee
import eemont  # noqa: F401
import geemap
import pandas as pd

### Authenticate Earth Engine

In [ ]:
# ee.Authenticate()
ee.Initialize()

## Data and Area of interest

### Get boundaries

In [ ]:
# Only available to Bison lab members
buffalo_expansion = ee.FeatureCollection(
    "projects/bison-lab/assets/Buffalo_Expansion_polygons"
)

In [ ]:
# Select specific boundaries from list
parcels = buffalo_expansion.filter(
    ee.Filter.inList(
        "Name",
        [
            "Shoshone Tribe",
            "Buffalo Initiative",
            "Hellyer Tribal Lease",
            "Adels Property",
            "Hoopengarner Property",
            "Hellyer Place",
        ],
    )
)

### Plot parcel boundaries

In [ ]:
m = geemap.Map()
m.addLayer(parcels, {"color": "red"}, "Parcels")
m.centerObject(parcels, 14)
m

## Load data

In [ ]:
# Get the outer bounds of the parcels
aoi = parcels.geometry().bounds()

In [ ]:
# Get timeseries from Dynamic World, a 10m near-real-time Land Use/Land Cover dataset
# that includes class probabilities and label information for nine classes
data = (
    ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
    .filterDate("2020-01-01", "2021-01-01")  # 1 year time range
    .filterBounds(aoi)
    .select("label")
)

In [ ]:
# Reduce the timeseries and assign the mode (most common) classification to each pixel
img = data.reduce(ee.Reducer.mode())

# Clip the extent of the image to the outer bounds of the parcels
img = img.clip(aoi)

In [ ]:
# Define legend colours
class_labels = [
    "water",
    "trees",
    "grass",
    "flooded_vegetation",
    "crops",
    "shrub_and_scrub",
    "built",
    "bare",
    "snow_and_ice",
]
class_index = [i for i in range(0, len(class_labels))]
palette = [
    "#419BDF",
    "#397D49",
    "#88B053",
    "#7A87C6",
    "#E49635",
    "#DFC35A",
    "#C4281B",
    "#A59B8F",
    "#B39FE1",
]
legend_dict = dict(zip(class_labels, palette))

In [ ]:
m = geemap.Map()
m.addLayer(
    img, {"min": 0, "max": len(palette), "palette": palette}, "Classified Composite"
)
m.addLayer(parcels, {"color": "black"}, "Parcels")
m.centerObject(aoi, 14)
m.add_legend(legend_dict=legend_dict)
m

## Compute frequency of each class

In [ ]:
stats = img.reduceRegions(
    collection=parcels, reducer=ee.Reducer.frequencyHistogram(), scale=10
)

In [ ]:
# convert to dataframe
df = geemap.ee_to_df(stats.select(["histogram", "Name"])).set_index("Name")

In [ ]:
# parse histogram to columns and normalize
df = df["histogram"].apply(pd.Series)
df = df.div(df.sum(axis="rows"), axis="columns")
# rename columns
df = df.rename(columns=dict((f"{i}", j) for i, j in zip(class_index, class_labels)))

In [ ]:
df.plot.bar();